# Social Network
Design a simplified social network that relies heavily on using different data structures. An idea adapted from [LeetCode's Design Twitter Problem](https://leetcode.com/problems/design-twitter/). The design of the system is leaved to the implementer's choice whether he adopts an OO (object oriented) style or the usual procedural one. However, following **the OO style involves bonus marks**!!



## Group Members:
• Omar Bahaa 120180027

• Abdelrahman Nawar 120180025

• Abdelrahman Khaled 120180041

• Martin Ehab 120180004

• Mohamed Hatem 120170012

### Initialization

In [ ]:
import networkx as nx
import collections
import random
import uuid 
from time import ctime

### Project Template
You may modify it (by applying some design pattern/s, for example) but maintain the same functionality.

In [ ]:
class User:
    def __init__(self, Name: str) -> None:
        """
        :param Name: Given name of the user
        """
        self.ID = SocialNetwork.generateID()
        SocialNetwork.IDs.append(self.ID)
        self.name = Name
        self.deque_of_tweets = collections.deque(maxlen=5)
        self.deque_of_activities = collections.deque(maxlen=5)
        SocialNetwork.users.add_node(self)

    def __repr__(self): # Omar Bahaa
        """
        :return: template representation for the object
        """
        name = "Name: " + str(self.name)
        user_id = "User ID: " + str(self.ID)
        return name + '\n' + user_id

    # ----------------------------------------------------------------- #
    def follow(self, followee) -> None: # Abdelrahman Sherbiny
        """
      Follower follows a followee.
      If the operation is invalid, it should be a no-op.
      """
        if (followee, self) not in SocialNetwork.users.edges:
            SocialNetwork.users.add_edge(followee, self)
            followee.deque_of_activities.appendleft(f"{self.name} is now following you")
        else:
            pass

    # ----------------------------------------------------------------- #
    def unfollow(self, followee) -> None: # Abdelrahman Sherbiny
        """
      Follower unfollows a followee.
      If the operation is invalid, it should be a no-op.
      """
        if (followee, self) in SocialNetwork.users.edges:
            SocialNetwork.users.remove_edge(followee, self)
        else:
            pass

    # ----------------------------------------------------------------- #
    def react(self, postId: str) -> None: # Abdelrahman Nawar
        """
      User reacts to a post at a certain time.
      If react() is called twice, the user un-reacts to the post.
      """
        list_of_reacts = SocialNetwork.tweets[postId][1:]
        tweet = SocialNetwork.tweets[postId][0]
        if self in list_of_reacts:
            list_of_reacts.remove(self)
        else:
            list_of_reacts.append(self)
            author = tweet.author
            author.deque_of_activities.appendleft(f"{self.name} reacted to your tweet.")
        SocialNetwork.tweets[postId] = [tweet] + list_of_reacts

    # ----------------------------------------------------------------- #
    def post(self, postBody): # Martin Ehab
        """
      User composes a new post with string body at a certain time.
      """
        tweet = Tweet(self, postBody)
        self.deque_of_tweets.appendleft(tweet)
        list_of_followers = self.news_to(self, [], 1)
        if list_of_followers:
            for user in set(list_of_followers):  # to apply for unique items only
                user.deque_of_activities.appendleft(f"{self.name} shared a tweet.")
                user.deque_of_tweets.appendleft(tweet)
        return tweet.ID

    # --- BONUS ------------------------------------------------------- #
    def share(self, postBody, users): # Abdelrahman Nawar
        """
        User shares a post to a group of users.
        Disallow sharing a post twice with a user where this post
        has been already shared with.
        """
        tweet = Tweet(self, postBody)
        self.deque_of_tweets.appendleft(tweet)
        list_of_followers = users
        for user in list_of_followers:
            user.deque_of_activities.appendleft(f"{self.name} shared a tweet with you.")
            user.deque_of_tweets.appendleft(tweet)
        return tweet.ID

    @staticmethod
    def news_to(node, list_of_users, count) -> list: # Abdelrahman Nawar
        """
        Helper function used to get follower users up to third degree to the input user (node)
        :param node: current user that we want to get their followers up to third degree
        :param list_of_users: list of followers up to third degree
        :param count: to keep track of the degrees to stop when we reach the third degree
        :return: list of Users
        """
        if count > 3:
            return list_of_users
        for v in SocialNetwork.users.neighbors(node):
            list_of_users.append(v)
            User.news_to(v, list_of_users, count + 1)
        return list_of_users

In [ ]:
class Tweet: # Omar Bahaa
    def __init__(self, User: User, postBody: str) -> None:
        """
        :param User: Author of the tweet
        :param postBody: Content of the tweet
        """
        self.ID = str(User.ID) + '-' + SocialNetwork.generateID()
        SocialNetwork.IDs.append(self.ID)
        self.postBody = postBody
        self.timestamp = ctime()
        self.author = User
        SocialNetwork.tweets[self.ID] = [self]

    def __repr__(self):
        """
        :return: template for how a Tweet object should be represented
        """
        breaker = "-" * 150
        author = "Author: " + str(self.author.name)
        at_time = "Tweet time: " + str(self.timestamp)
        body = "Tweet:\n\n" + self.postBody
        ID = "Tweet ID: " + self.ID
        return breaker + '\n' + author + '\n' + at_time + '\n' + ID + '\n' + body + '\n' + breaker

In [ ]:
class SocialNetwork:
    users = nx.DiGraph()
    tweets = {}
    IDs = []

    def __new__(cls, *args, **kwargs): # Omar Bahaa
        """
        Applying of Singleton design pattern to avoid the creation of more than one instance
        """
        if not hasattr(cls, 'instance'):
            cls.instance = super(SocialNetwork, cls).__new__(cls)
        return cls.instance

    @staticmethod
    def generateID() -> str: # Omar Bahaa
        """
        :return: unique ID to be used by User and Tweet classes
        """
        ID = str(uuid.uuid4()).replace("-", "")
        if ID in SocialNetwork.IDs:
            ID = SocialNetwork.generateID()
        return ID

    # ----------------------------------------------------------------- #
    def getNewsFeed(self, user: User): # Mohamed Hatem
        """
        Retrieve the 5 most recent posts in the user's news feed.
        Each item in the news feed must be posted by
        the user himself, user's followees (1st degree relation)
        or followees of followees up to 3rd degree. Also, (if implemented)
        include shared posts with a user in his/her news feed.
        Posts must be ordered from most recent to least recent.
        """
        print("*"*50 + f"{user.name}'s feed list" + "*"*50)
        if user.deque_of_tweets:
          for i in user.deque_of_tweets:
            print(i)

    # --- BONUS ------------------------------------------------------- #
    def getNotifications(self, user: User): # Mohamed Hatem
        """
        Retrieve the 5 most recent user's notifications.
        When user A reacts to post of user B or shares a post
        with user B, this accounts for notification for user B.
        Notifications must be ordered from most recent to least recent.
        """
        print("*"*50 + f"{user.name}'s notifications list" + "*"*50)
        if user.deque_of_activities:
          for i in user.deque_of_activities:
            print(i,'\n')

    # --- BONUS ------------------------------------------------------- #
    def search(self, userName: str) -> list or str: # Abdelrahman Nawar
        """
        Provided certain name, search for the matching users names.
        """
        possibilities = userName.split(" ")
        for i in range(len(possibilities)):
            possibilities.append(possibilities[i][:(len(possibilities[i]) // 2) + 1])
        'possibilities contains the given userName (first and last) and the first half of each name'
        results = []
        for user in self.users.nodes:
            for possibility in possibilities:
                if possibility.lower() in user.name.lower():  # to make it case insensitive
                    results.append(user)
                    break  # to avoid repetitions
        if results:
            return results
        return "No results found"

# Testing

In [ ]:
SN = SocialNetwork()
user1 = User("user1")
user2 = User("user2")
user3 = User("user3")
user4 = User("user4")
user5 = User("user5")
user5.follow(user1)
user4.follow(user5)
post1 = user1.post("THIS IS A DEMO ")
SN.getNewsFeed(user5)
print("\n")
SN.getNewsFeed(user4)
user3.react(post1)
print("\n")
SN.getNotifications(user1)

**************************************************user5's feed list**************************************************
------------------------------------------------------------------------------------------------------------------------------------------------------
Author: user1
Tweet time: Sun Feb 14 10:57:11 2021
Tweet ID: 799b86d4b9e3444c9c073eba118b6362-5f5240c3a32d4875a59618315915769f
Tweet:

THIS IS A DEMO 
------------------------------------------------------------------------------------------------------------------------------------------------------


**************************************************user4's feed list**************************************************
------------------------------------------------------------------------------------------------------------------------------------------------------
Author: user1
Tweet time: Sun Feb 14 10:57:11 2021
Tweet ID: 799b86d4b9e3444c9c073eba118b6362-5f5240c3a32d4875a59618315915769f
Tweet:

THIS IS A DEMO 
-------